In [5]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [6]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [7]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2_sup")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [9]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
301,model_24_4_24,1.000000,0.687622,1.000000,1.0,1.000000,1.147849e-08,0.185441,2.427694e-08,4.288182e-08,3.357938e-08,0.000046,0.000107,1.0000,0.000112,118.565582,168.539491,"Hidden Size=[10], regularizer=0.05, learning_r..."
302,model_24_4_23,1.000000,0.687619,1.000000,1.0,1.000000,1.162496e-08,0.185443,2.337724e-08,4.355798e-08,3.346761e-08,0.000046,0.000108,1.0000,0.000112,118.540223,168.514131,"Hidden Size=[10], regularizer=0.05, learning_r..."
304,model_24_4_22,1.000000,0.687617,1.000000,1.0,1.000000,1.186419e-08,0.185444,2.249223e-08,4.460235e-08,3.356086e-08,0.000047,0.000109,1.0000,0.000114,118.499482,168.473390,"Hidden Size=[10], regularizer=0.05, learning_r..."
306,model_24_4_21,1.000000,0.687614,1.000000,1.0,1.000000,1.225407e-08,0.185446,2.138803e-08,4.648996e-08,3.393899e-08,0.000048,0.000111,1.0000,0.000115,118.434815,168.408724,"Hidden Size=[10], regularizer=0.05, learning_r..."
307,model_24_4_20,1.000000,0.687613,1.000000,1.0,1.000000,1.262624e-08,0.185447,2.042298e-08,4.776828e-08,3.409563e-08,0.000049,0.000112,1.0000,0.000117,118.374978,168.348887,"Hidden Size=[10], regularizer=0.05, learning_r..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3894,model_10_8_10,0.988104,0.731550,0.999333,1.0,0.999505,7.061941e-03,0.159363,3.401326e-04,3.180583e-14,1.700663e-04,0.027112,0.084035,1.0571,0.087613,67.906071,103.253470,"Hidden Size=[7], regularizer=0.2, learning_rat..."
3895,model_10_8_7,0.988104,0.731550,0.999333,1.0,0.999505,7.061941e-03,0.159363,3.401326e-04,3.180583e-14,1.700663e-04,0.027112,0.084035,1.0571,0.087613,67.906071,103.253470,"Hidden Size=[7], regularizer=0.2, learning_rat..."
3896,model_10_8_22,0.988104,0.731550,0.999333,1.0,0.999505,7.061941e-03,0.159363,3.401326e-04,3.180583e-14,1.700663e-04,0.027112,0.084035,1.0571,0.087613,67.906071,103.253470,"Hidden Size=[7], regularizer=0.2, learning_rat..."
3897,model_10_8_13,0.988104,0.731550,0.999333,1.0,0.999505,7.061941e-03,0.159363,3.401326e-04,3.180583e-14,1.700663e-04,0.027112,0.084035,1.0571,0.087613,67.906071,103.253470,"Hidden Size=[7], regularizer=0.2, learning_rat..."
